In [1]:
pip install autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.5/573.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 924.6/924.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 115.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 41.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: te

In [2]:
import tensorflow as tf
import autokeras as ak

Using TensorFlow backend


# load california housing price

## prediction dataset

In [18]:
from sklearn.datasets import fetch_california_housing

house_dataset = fetch_california_housing()

# import pandas
import pandas as pd

# Extract feature
data = pd.DataFrame(house_dataset.data, columns=house_dataset.feature_names)

# Extract target
target = pd.Series(house_dataset.target, name="MEDV")

from sklearn.model_selection import train_test_split

train_data, test_data, train_targets, test_targets = train_test_split(
    data, target, test_size = 0.2, random_state=42
)

In [19]:
# Check the shape of whole dataset and the splited training and test set
print("--Shape of the whole data--\n {}".format(data.shape))
print("\n--Shape of the target vector--\n {}".format(target.shape))
print("\n--Shape of the training data--\n {}".format(train_data.shape))
print("\n--Shape of the testing data--\n {}".format(test_data.shape))


--Shape of the whole data--
 (20640, 8)

--Shape of the target vector--
 (20640,)

--Shape of the training data--
 (16512, 8)

--Shape of the testing data--
 (4128, 8)


# Run The StructuredDataReressor

In [20]:
regressor = ak.StructuredDataRegressor(max_trials=10, overwrite=True)
regressor.fit(x=train_data, y=train_targets, batch_size=1024)

Trial 10 Complete [00h 01m 07s]
val_loss: 0.3209627866744995

Best val_loss So Far: 0.31974008679389954
Total elapsed time: 00h 14m 25s
Epoch 1/164
17/17 [==============================] - 2s 10ms/step - loss: 2.7677 - mean_squared_error: 2.7677
Epoch 2/164
17/17 [==============================] - 0s 10ms/step - loss: 1.0485 - mean_squared_error: 1.0485
Epoch 3/164
17/17 [==============================] - 0s 13ms/step - loss: 0.7821 - mean_squared_error: 0.7821
Epoch 4/164
17/17 [==============================] - 0s 16ms/step - loss: 0.6528 - mean_squared_error: 0.6528
Epoch 5/164
17/17 [==============================] - 0s 18ms/step - loss: 0.5688 - mean_squared_error: 0.5688
Epoch 6/164
17/17 [==============================] - 0s 19ms/step - loss: 0.5175 - mean_squared_error: 0.5175
Epoch 7/164
17/17 [==============================] - 0s 17ms/step - loss: 0.4875 - mean_squared_error: 0.4875
Epoch 8/164
17/17 [==============================] - 0s 18ms/step - loss: 0.4677 - mean_square

# Predicted best model & evaluate the best models on the test data

In [21]:
predicted_y = regressor.predict(test_data)
print(predicted_y)

test_loss, test_mse = regressor.evaluate(test_data, test_targets, verbose=0)
print("Test MSE: ", test_mse)

129/129 [==============================] - 0s 2ms/step
[[0.5172578 ]
 [0.9159235 ]
 [4.767477  ]
 ...
 [4.8578615 ]
 [0.67431414]
 [1.8568442 ]]
Test MSE:  0.30072858929634094


# Multi label

In [23]:
from sklearn.datasets import make_multilabel_classification

X, Y = make_multilabel_classification(
    n_samples=100,
    n_features=64,
    n_classes=3,
    n_labels=2,
    allow_unlabeled=False,
    random_state=1,
)
X = X.reshape((100, 8, 8))
X.shape, Y.shape


((100, 8, 8), (100, 3))

In [24]:
x_train, x_test, y_train, y_test = X[:80], X[80:], Y[:80], Y[80:]

## Run the Image classifier for multi-label classification

In [26]:
# Initialize the image classifier.
clf = ak.ImageClassifier(
    max_trials=3, multi_label=True, overwrite=True
)  # It tries two different pipelines.

# Feed the image classifier with training data
# 20% of the data is used as validation data by default for tuning
# the process may run for a bit long time, please try to use GPU
clf.fit(x_train, y_train, epochs=3, verbose=2)  # each model is trained for three epochs


Trial 3 Complete [00h 03m 33s]
val_loss: 0.7258502244949341

Best val_loss So Far: 0.5857412815093994
Total elapsed time: 00h 04m 36s
Epoch 1/3
3/3 - 2s - loss: 0.7219 - accuracy: 0.4125 - 2s/epoch - 662ms/step
Epoch 2/3
3/3 - 0s - loss: 0.6073 - accuracy: 0.4125 - 23ms/epoch - 8ms/step
Epoch 3/3
3/3 - 0s - loss: 0.5864 - accuracy: 0.3250 - 23ms/epoch - 8ms/step


### predict the best model

In [27]:
predicted_y = clf.predict(x_test)
print("The prediction shape is: {}".format(predicted_y.shape))
print(
    "The predicted labels of the first five instances are:\n {}".format(
        predicted_y[:5, :]
    )
)

1/1 [==============================] - 0s 10ms/step
The prediction shape is: (20, 3)
The predicted labels of the first five instances are:
 [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [28]:
test_loss, test_acc = clf.evaluate(x_test, y_test, verbose=0)
print("Test accuracy: ", test_acc)

Test accuracy:  0.4000000059604645
